In [1]:
import sys 
import pandas as pd
import logging

sys.path.append('../')

import utils.llm 
import utils.scoring

logger = logging.getLogger()
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

class Config:
    DATA_DIR = '../construct_dataset/'
    DATA_OUT_DIR = '../data_out/'
    TEMPERATURE = 0.8
    SAMPLE_N = 30
    # 収集対象の対象クイズの範囲
    TARGET_QUIZ_RANGE = [i for i in range(1, 201)]
    


annotated_df = pd.read_csv(Config.DATA_DIR + 'df_JAQKET_qa_annot1.csv')





Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class y

In [2]:
llm = utils.llm.LLM()


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ryoryon66/.pyenv/versions/3.9.8/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA SETUP: WARNING! libcuda.so not found! Do you have a CUDA driver installed? If you are on a cluster, make sure you are on a CUDA machine!
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Loading binary /home/ryoryon66/.pyenv/versions/3.9.8/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/ryoryon66/.pyenv/versions/3.9.8/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: No GPU detected! Check your CUDA paths. Proceeding to load CPU-only library...
  warn(msg)


In [3]:
def sampling(query:str,temperature:float,sample_n:int) -> list[str]:
    
    sampled = []
    
    for i in range(sample_n):
        
        q = query
        sampled.append(llm.generate(q,temperature))
    
    logger.info("sampled: {}".format(sampled))
    
    return sampled


In [4]:
# serial_number, qid, query, responseを記録するdf

df = pd.DataFrame(columns=['serial_number','qid','query','sample'])


serial_number = 0



for i in Config.TARGET_QUIZ_RANGE:
    
    # show progress
    if i % 10 == 0:
        logger.info("now processing the quiz of index... {}".format(i))
    
    qid = annotated_df.iloc[i]['qid']
    query = annotated_df.iloc[i]['question']
    response = annotated_df.iloc[i]['generated_answer']
    
    sampled = sampling(query,Config.TEMPERATURE,Config.SAMPLE_N)
    
    for sample in sampled:
        
        #  The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
        
        df = pd.concat([df,pd.DataFrame([[serial_number,qid,query,sample]],columns=['serial_number','qid','query','sample'])])
        
        serial_number += 1


sampled: ['『東方見聞録』です。', '星の王子さま。この作品は、児童書出版社によって出版されました。作家のアントワーヌ・ド・サン=テグジュペリは、飛行士であり、パイロットでもありました。彼は、サハラ砂漠やインドの砂漠を旅し、そこで出会ったさまざまな民族との交流を通じて世界を広く知る旅をしました。この作品では、サハラ砂漠に住むトゥファと呼ばれる民族との出会いにより、王子さまは初めて地球に降り立ちます。そして、彼の友人とともに「大いなる星」を見つけます。それは、地球上で最も美しいとされる宝石がいっぱいの惑星で、王子さまと友人は、その美しさに感動し、やがて地球', '『マルコ・ポーロ 東方見聞録』。この作品は、ルスティケロの最も有名な著作であり、マルコ・ポーロの中国での体験についての小説です。', '黄金の島 イタリア/スペイン物語 イタリア半島 地中海と大西洋岸沿いに広がった地域にある島々。ポーロはスペインに長く滞在し、多くの文学や工芸品を学んだ。スペインやイタリア半島は文化の中心となり、南欧の美しい国々と交流した。この本には、イタリア半島の多くの町や都市が描かれ、ポーロが訪れた当時の情景を生き生きと描き出している。 イタリア半島 イタリア半島 イタリア南部の海岸線 イタリア北部 イタリア半島 イタリア中部 イタリア南部 イタリア北東部 イタリア中部 イタリア北西部 イタリア北東部 イタリア南西部 イタリア中部 イタリア北部 イタリア北東部 イタリア中部 イタリア北東部 イタリア北東部 イタリア中部 イタリア北東部 イタリア中部 イタリア北東部 イタリア', 'レオナルド・ダ・ヴィンチの「東方見聞録」。多くの文化や国々を旅したマルコによる記録である。その中には、中国やインドを訪れた際に出会った人々も多く含まれている。彼はまた、モンゴルや東欧の国々を訪れた際の旅行記も残している。これらの地域にはまだ多くの文化的・歴史的なつながりがあり、マルコはそれらを訪れることで、それらの国の独自の伝統や文化を探究し、ヨーロッパとは異なる世界や社会を紹介した。', '『ザ・ミッション』です。彼はヴェネツィアと中国に長い航海を行った探検家や地理学者であり、またイタリア人として広く認知された人物の一人でもあります。彼の作品は高く評価され、多くの賞を受賞しています。『ザ・ミッション

In [5]:
df.shape

(6000, 4)

In [6]:
import datetime

info = "temperature_{}_sample_n_{}".format(Config.TEMPERATURE,Config.SAMPLE_N)
suffix = datetime.datetime.now().strftime('%m_%d_%H_%M_%S')

df.to_csv(Config.DATA_OUT_DIR + 'same_query_{}_{}.csv'.format(info,suffix),index=False)




In [7]:
df

,serial_number,qid,query,sample
0,0,QA20CAPR-0003,作家のルスティケロが、マルコ・ポーロから聞いた話をまとめた作品といえば何でしょう?,『東方見聞録』です。
0,1,QA20CAPR-0003,作家のルスティケロが、マルコ・ポーロから聞いた話をまとめた作品といえば何でしょう?,星の王子さま。この作品は、児童書出版社によって出版されました。作家のアントワーヌ・ド・サン=...
0,2,QA20CAPR-0003,作家のルスティケロが、マルコ・ポーロから聞いた話をまとめた作品といえば何でしょう?,『マルコ・ポーロ 東方見聞録』。この作品は、ルスティケロの最も有名な著作であり、マルコ・ポー...
0,3,QA20CAPR-0003,作家のルスティケロが、マルコ・ポーロから聞いた話をまとめた作品といえば何でしょう?,黄金の島 イタリア/スペイン物語 イタリア半島 地中海と大西洋岸沿いに広がった地域にある島々...
0,4,QA20CAPR-0003,作家のルスティケロが、マルコ・ポーロから聞いた話をまとめた作品といえば何でしょう?,レオナルド・ダ・ヴィンチの「東方見聞録」。多くの文化や国々を旅したマルコによる記録である。そ...
...,...,...,...,...
0,5995,QA20CAPR-0405,味噌や醤油の原料になる六条と、ビールやウイスキーの原料になる二条に大別されるイネ科の植物は何...,コウライドロコシアブラガク茎、別名「花コウライ」と呼ばれる。
0,5996,QA20CAPR-0405,味噌や醤油の原料になる六条と、ビールやウイスキーの原料になる二条に大別されるイネ科の植物は何...,米麦芽
0,5997,QA20CAPR-0405,味噌や醤油の原料になる六条と、ビールやウイスキーの原料になる二条に大別されるイネ科の植物は何...,コウライテンダネード コウライテンダネードとは、東南アジア原産の稲の一種であり、日本の米に非...
0,5998,QA20CAPR-0405,味噌や醤油の原料になる六条と、ビールやウイスキーの原料になる二条に大別されるイネ科の植物は何...,米コウライ属の一種。米コウライは米コウラとも表記されます。コウライは東アジア原産で、日本、中...
